# D-optimal experiment design: comparing ABPG and Frank-Wolfe
Solve the D-Optimal experiment design problem
$$
\begin{array}{ll}
\textrm{minimize}   & F(x):=\log\left(\det\left(\sum_{i=1}^n x_i V_i V_i^T\right)\right) \\
\textrm{subject to} & \sum_{i=1}^n x_i = 1, \\ 
                    & x_i\geq 0, \quad i=1,\ldots,n
\end{array}
$$
where $V_i\in R^m$ for $i=1,\ldots,n$.

Methods compared:
* Original Frank-Wolfe method
* Frank-Wolfe method with away steps
* Bregman Proximal Gradient (BPG) method with adaptive line search
* Accelerated Bregman Proximal Gradient (ABPG) method with gain adaption

In [1]:
cd  C:\\github\accbpg

C:\github\accbpg


In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams.update({'font.size': 12, 'font.family': 'serif'})
#matplotlib.rcParams.update({'text.usetex': True})

import accbpg

In [3]:
m = 80
n = [100, 200, 300, 400, 600, 800, 1000]
K = 10
eps = ['1e-3', '1e-4', '1e-5', '1e-6', '1e-7', '1e-8']

Nmax = 100000
Nskip = int(Nmax/10)

Ieps = dict()
Teps = dict()
for s in eps:
    Ieps[s] = np.zeros((4, len(n), K))
    Teps[s] = np.zeros((4, len(n), K))

for i in range(len(n)):
    print("\n********** m = {0:d}, n = {1:d} **********".format(m, n[i]))
    for k in range(K):
        f, h, L, x0Kh = accbpg.D_opt_design(m, n[i])
        x0KY = accbpg.D_opt_KYinit(f.H)
        x0Mx = (1-1e-3)*x0KY + 1e-3*x0Kh

        _, F_FWKY, _, _, T_FWKY = accbpg.D_opt_FW(f.H, x0KY, 1e-8, maxitrs=Nmax, verbskip=Nskip)
        _, F_WAKY, _, _, T_WAKY = accbpg.D_opt_FW_away(f.H, x0KY, 1e-8, maxitrs=Nmax, verbskip=Nskip)
        _, F_LSKh, _, T_LSKh = accbpg.BPG(f, h, L, x0Kh, maxitrs=Nmax, linesearch=True, ls_ratio=1.5, verbskip=Nskip)
        _, F_ABKh, _, _, _, T_ABKh = accbpg.ABPG_gain(f, h, L, x0Kh, gamma=2, maxitrs=Nmax, ls_inc=1.5, ls_dec=1.5, restart=True, verbskip=Nskip)

        Fmin = min(F_FWKY.min(), F_WAKY.min(), F_LSKh.min(), F_ABKh.min())
        F = [F_FWKY, F_WAKY, F_LSKh, F_ABKh]
        T = [T_FWKY, T_WAKY, T_LSKh, T_ABKh]
        for s in eps:
            for j in range(len(F)):
                I_eps = np.nonzero(F[j] - Fmin <= float(s))
                if len(I_eps[0]) > 0:
                    i_eps = I_eps[0][0]
                    t_eps = T[j][i_eps]
                else:
                    i_eps = Nmax + 1
                    t_eps = T[j][-1]
                Ieps[s][j,i,k] = i_eps
                Teps[s][j,i,k] = t_eps


********** m = 80, n = 100 **********

Solving D-opt design problem using Frank-Wolfe method
     k      F(x)     pos_slack   neg_slack    time
     0   4.670e+01   1.524e-01   2.329e-01     0.0
 10000   4.643e+01   8.287e-04   5.726e-02     0.9
 20000   4.643e+01   3.262e-04   2.476e-02     1.7
 30000   4.643e+01   1.397e-04   1.110e-02     2.6
 40000   4.643e+01   6.269e-05   5.055e-03     3.4
 50000   4.643e+01   2.849e-05   2.317e-03     4.2
 60000   4.643e+01   1.347e-05   1.065e-03     5.1
 70000   4.643e+01   6.002e-06   4.906e-04     5.9
 80000   4.643e+01   2.754e-06   2.261e-04     6.7
 90000   4.643e+01   1.294e-06   1.042e-04     7.5

Solving D-opt design problem using Frank-Wolfe method with away steps
     k      F(x)     pos_slack   neg_slack    time
     0   4.670e+01   1.524e-01   2.329e-01     0.0

BPG_LS method for min_{x in C} F(x) = f(x) + Psi(x)
     k      F(x)         Lk       time
     0   4.670e+01   6.667e-01     0.0

ABPG_gain method for min_{x in C} F(x) =


Solving D-opt design problem using Frank-Wolfe method
     k      F(x)     pos_slack   neg_slack    time
     0   4.696e+01   1.351e-01   2.598e-01     0.0
 10000   4.673e+01   1.142e-03   8.463e-02     0.8
 20000   4.672e+01   5.001e-04   4.059e-02     1.7
 30000   4.672e+01   2.432e-04   2.048e-02     2.5
 40000   4.672e+01   1.200e-04   1.058e-02     3.3
 50000   4.672e+01   6.509e-05   5.536e-03     4.1
 60000   4.672e+01   3.320e-05   2.914e-03     4.9
 70000   4.672e+01   1.749e-05   1.538e-03     5.7
 80000   4.672e+01   9.258e-06   8.134e-04     6.6
 90000   4.672e+01   4.745e-06   4.304e-04     7.4

Solving D-opt design problem using Frank-Wolfe method with away steps
     k      F(x)     pos_slack   neg_slack    time
     0   4.696e+01   1.351e-01   2.598e-01     0.0

BPG_LS method for min_{x in C} F(x) = f(x) + Psi(x)
     k      F(x)         Lk       time
     0   4.696e+01   6.667e-01     0.0

ABPG_gain method for min_{x in C} F(x) = f(x) + Psi(x)
     k      F(x)       t

 90000   1.706e+01   1.975e-01   186.7

ABPG_gain method for min_{x in C} F(x) = f(x) + Psi(x)
     k      F(x)       theta         Gk         TSG       D(x+,y)     D(z+,z)      Gavg       time
     0   1.859e+01   1.000e+00   6.667e-01   1.000e+00   5.419e-01   5.419e-01   8.165e-01     0.0
 10000   1.706e+01   5.088e-04   4.444e-01   4.432e-01   1.998e-15   1.741e-08   3.938e-01    27.8
 20000   1.706e+01   5.193e-04   4.444e-01   5.763e-01   4.441e-16   2.858e-09   4.677e-01    55.8
 30000   1.706e+01   8.402e-04   1.975e-01   7.839e-01   1.332e-15   2.408e-09   5.013e-01    83.5
 40000   1.706e+01   1.204e-03   8.779e-02   9.627e-01   7.327e-15   5.249e-09   5.179e-01   111.2

Solving D-opt design problem using Frank-Wolfe method
     k      F(x)     pos_slack   neg_slack    time
     0   2.350e+01   2.105e+00   2.633e-01     0.0
 10000   1.697e+01   1.918e-03   1.367e-01     0.9
 20000   1.696e+01   1.031e-03   1.139e-01     1.8
 30000   1.696e+01   6.532e-04   1.022e-01     2.7
 

 10000   1.717e+01   4.309e-04   4.444e-01   7.646e-01   1.155e-14   8.134e-08   3.531e-01    27.8
 20000   1.717e+01   4.802e-04   4.444e-01   9.308e-01   2.220e-15   1.034e-08   4.071e-01    55.6
 30000   1.717e+01   9.503e-04   1.317e-01   9.254e-01   3.997e-15   4.783e-09   4.371e-01    84.3

Solving D-opt design problem using Frank-Wolfe method
     k      F(x)     pos_slack   neg_slack    time
     0   2.340e+01   1.378e+00   2.868e-01     0.0
 10000   1.778e+01   2.115e-03   1.879e-01     1.0
 20000   1.777e+01   1.093e-03   1.694e-01     1.9
 30000   1.777e+01   8.533e-04   1.591e-01     2.9
 40000   1.776e+01   6.783e-04   1.519e-01     3.9
 50000   1.776e+01   6.017e-04   1.466e-01     4.8
 60000   1.776e+01   5.145e-04   1.421e-01     5.8
 70000   1.776e+01   4.659e-04   1.385e-01     6.8
 80000   1.776e+01   4.125e-04   1.353e-01     8.0
 90000   1.776e+01   3.843e-04   1.325e-01     9.0

Solving D-opt design problem using Frank-Wolfe method with away steps
     k      F(x)

 60000   7.723e+00   1.975e-01   161.6
 70000   7.723e+00   1.317e-01   188.3
 80000   7.723e+00   8.779e-02   215.8
 90000   7.723e+00   2.963e-01   242.3

ABPG_gain method for min_{x in C} F(x) = f(x) + Psi(x)
     k      F(x)       theta         Gk         TSG       D(x+,y)     D(z+,z)      Gavg       time
     0   1.102e+01   1.000e+00   6.667e-01   1.000e+00   5.224e-01   5.224e-01   8.165e-01     0.0
 10000   7.723e+00   7.470e-04   1.317e-01   3.512e-01   1.095e-13   5.585e-07   2.666e-01    34.5
 20000   7.723e+00   3.751e-04   4.444e-01   1.163e-01   4.441e-16   2.713e-08   2.748e-01    72.2
 30000   7.723e+00   9.832e-04   2.963e-01   1.042e+00   1.799e-14   1.785e-08   3.493e-01   110.0

Solving D-opt design problem using Frank-Wolfe method
     k      F(x)     pos_slack   neg_slack    time
     0   1.886e+01   2.209e+00   2.912e-01     0.0
 10000   7.990e+00   2.647e-03   2.437e-01     0.9
 20000   7.968e+00   1.591e-03   2.281e-01     1.9
 30000   7.958e+00   1.231e-03   2

...

 70000  -4.133e+00   1.317e-01   418.1
 80000  -4.133e+00   8.779e-02   474.2
 90000  -4.133e+00   1.317e-01   530.5

ABPG_gain method for min_{x in C} F(x) = f(x) + Psi(x)
     k      F(x)       theta         Gk         TSG       D(x+,y)     D(z+,z)      Gavg       time
     0   3.639e+00   1.000e+00   6.667e-01   1.000e+00   2.084e-01   2.084e-01   8.165e-01     0.0
 10000  -4.133e+00   4.660e-04   1.975e-01   7.702e-01   1.863e-12   1.114e-05   1.862e-01    67.3
 20000  -4.133e+00   4.772e-04   1.317e-01   7.657e-01   2.760e-13   1.583e-06   1.864e-01   135.0
 30000  -4.133e+00   4.483e-04   1.317e-01   9.061e-01   7.661e-14   4.206e-07   1.873e-01   204.2
 40000  -4.133e+00   2.618e-04   4.444e-01   7.264e-01   9.770e-15   1.962e-07   1.904e-01   272.8
 50000  -4.133e+00   5.213e-04   1.975e-01   2.535e-01   4.441e-15   6.446e-08   2.013e-01   340.1
 60000  -4.133e+00   7.262e-04   1.975e-01   7.916e-01   4.352e-14   1.042e-07   2.193e-01   408.9
 70000  -4.133e+00   8.395e-04   8.

 20000  -3.860e+00   4.164e-04   1.975e-01   8.077e-01   2.509e-13   1.791e-06   1.804e-01   134.9
 30000  -3.860e+00   4.287e-04   1.317e-01   9.181e-01   1.819e-13   1.078e-06   1.751e-01   202.6
 40000  -3.860e+00   5.517e-04   1.975e-01   2.392e-01   6.217e-15   8.541e-08   1.768e-01   270.5
 50000  -3.860e+00   4.227e-04   2.963e-01   8.197e-01   2.975e-14   2.032e-07   1.833e-01   338.3

Solving D-opt design problem using Frank-Wolfe method
     k      F(x)     pos_slack   neg_slack    time
     0   1.599e+01   2.318e+00   2.340e-01     0.0
 10000  -2.771e+00   4.208e-03   2.480e-01     1.0
 20000  -2.825e+00   2.361e-03   2.392e-01     1.9
 30000  -2.847e+00   1.685e-03   2.351e-01     3.0
 40000  -2.859e+00   1.410e-03   2.324e-01     4.0
 50000  -2.867e+00   1.165e-03   2.310e-01     4.9
 60000  -2.873e+00   9.661e-04   2.295e-01     6.0
 70000  -2.877e+00   8.515e-04   2.284e-01     7.0
 80000  -2.881e+00   7.797e-04   2.277e-01     8.0
 90000  -2.883e+00   6.939e-04   2.271e

 30000  -2.883e+00   6.483e-04   5.853e-02   8.561e-01   2.964e-13   8.238e-07   1.757e-01   202.4
 40000  -2.883e+00   6.313e-04   8.779e-02   9.276e-01   9.082e-14   2.456e-07   1.781e-01   269.7
 50000  -2.883e+00   1.047e-03   2.963e-01   2.453e-01   1.554e-15   5.781e-09   2.001e-01   337.3

********** m = 80, n = 1000 **********

Solving D-opt design problem using Frank-Wolfe method
     k      F(x)     pos_slack   neg_slack    time
     0   1.376e+01   2.037e+00   2.601e-01     0.0
 10000  -6.521e+00   4.596e-03   2.506e-01     1.0
 20000  -6.582e+00   2.646e-03   2.439e-01     2.0
 30000  -6.606e+00   1.853e-03   2.414e-01     3.1
 40000  -6.620e+00   1.516e-03   2.399e-01     4.1
 50000  -6.629e+00   1.219e-03   2.388e-01     5.1
 60000  -6.635e+00   1.037e-03   2.379e-01     6.1
 70000  -6.640e+00   9.216e-04   2.373e-01     7.1
 80000  -6.644e+00   8.069e-04   2.368e-01     8.1
 90000  -6.647e+00   7.271e-04   2.363e-01     9.1

Solving D-opt design problem using Frank-Wolfe

 10000  -4.588e+00   5.075e-04   1.317e-01   3.908e-01   7.896e-13   7.843e-06   1.553e-01    81.1
 20000  -4.588e+00   3.454e-04   1.975e-01   1.002e+00   4.956e-13   4.146e-06   1.555e-01   161.7
 30000  -4.588e+00   3.227e-04   1.975e-01   8.001e-01   1.428e-13   1.714e-06   1.553e-01   242.5
 40000  -4.588e+00   3.426e-04   1.975e-01   5.337e-01   1.932e-14   3.083e-07   1.562e-01   323.2
 50000  -4.588e+00   7.796e-04   8.779e-02   3.809e-01   3.308e-14   1.429e-07   1.612e-01   404.1
 60000  -4.588e+00   6.729e-05   1.139e+01   0.000e+00   0.000e+00   5.358e-10   1.695e-01   486.7

Solving D-opt design problem using Frank-Wolfe method
     k      F(x)     pos_slack   neg_slack    time
     0   1.619e+01   2.461e+00   2.399e-01     0.0
 10000  -5.185e+00   4.727e-03   3.331e-01     1.0
 20000  -5.252e+00   2.680e-03   3.277e-01     2.0
 30000  -5.279e+00   2.007e-03   3.252e-01     3.0
 40000  -5.294e+00   1.498e-03   3.239e-01     4.0
 50000  -5.304e+00   1.267e-03   3.227e-01   

 10000  -6.189e+00   5.853e-02    67.5
 20000  -6.191e+00   8.779e-02   135.0
 30000  -6.191e+00   5.853e-02   202.5
 40000  -6.192e+00   1.317e-01   270.0
 50000  -6.192e+00   5.853e-02   337.3
 60000  -6.192e+00   5.853e-02   411.1
 70000  -6.192e+00   1.317e-01   482.6
 80000  -6.192e+00   5.853e-02   550.9
 90000  -6.192e+00   5.853e-02   618.5

ABPG_gain method for min_{x in C} F(x) = f(x) + Psi(x)
     k      F(x)       theta         Gk         TSG       D(x+,y)     D(z+,z)      Gavg       time
     0   3.085e+00   1.000e+00   6.667e-01   1.000e+00   1.799e-01   1.799e-01   8.165e-01     0.0
 10000  -6.192e+00   5.717e-04   1.317e-01   6.820e-01   1.983e-12   8.898e-06   1.221e-01    80.5
 20000  -6.192e+00   4.984e-04   8.779e-02   7.364e-01   3.213e-13   1.756e-06   1.201e-01   166.5
 30000  -6.192e+00   4.251e-04   8.779e-02   9.062e-01   2.189e-13   1.337e-06   1.171e-01   248.5
 40000  -6.192e+00   4.456e-04   8.779e-02   2.953e-02   1.776e-15   3.030e-07   1.164e-01   332.5

In [4]:
m = 80
n = [100, 200, 300, 400, 600, 800, 1000]
K = 10
eps = ['1e-3', '1e-4', '1e-5', '1e-6', '1e-7', '1e-8']

Nmax = 100000
Nskip = int(Nmax/10)

In [5]:
s = '1e-4'
meantype = 'g'
outliers = False

ns = np.array(n)
Igem = np.zeros((4,len(ns)))
Imax = np.zeros((4,len(ns)))
Imin = np.zeros((4,len(ns)))
Tgem = np.zeros((4,len(ns)))
Tmax = np.zeros((4,len(ns)))
Tmin = np.zeros((4,len(ns)))

for i in range(4):
    for j in range(len(ns)):
        idx = np.nonzero(Teps[s][i,j] > 0)
        #idx = np.nonzero(Ieps[s][i,j] < Nmax)
        if outliers == False and len(idx[0]) > 1: 
            Ieps_sij = Ieps[s][i,j][idx]
            Teps_sij = Teps[s][i,j][idx]
        else:
            Ieps_sij = Ieps[s][i,j]
            Teps_sij = Teps[s][i,j]
            
        if meantype == 'g':
            Igem[i,j] = Ieps_sij.prod()**(1.0/len(Ieps_sij))
            Imax[i,j] = Ieps_sij.max() 
            Imin[i,j] = Ieps_sij.min()
            Tgem[i,j] = Teps_sij.prod()**(1.0/len(Teps_sij))
            Tmax[i,j] = Teps_sij.max()
            Tmin[i,j] = Teps_sij.min()
        else: 
            Igem[i,j] = Ieps_sij.sum()/len(Ieps_sij)
            Imax[i,j] = Igem[i,j] + Ieps_sij.std() 
            Imin[i,j] = Igem[i,j] - Ieps_sij.std()
            Tgem[i,j] = Teps_sij.sum()/len(Teps_sij)
            Tmax[i,j] = Tgem[i,j] + Teps_sij.std()
            Tmin[i,j] = Tgem[i,j] - Teps_sij.std()
               
# Plot required number of iterations and time
matplotlib.rcParams.update({'font.size': 14, 'font.family': 'serif'})
matplotlib.rcParams.update({'text.usetex': True})

plt.subplots(1, 2, figsize=(10, 4))

labels = [r"FW", r"FW-away", r"BPG-LS", r"ABPG-g"]
linestyles = ['g-', 'k-.', 'b:', 'r--']

ax1 = plt.subplot(1,2,1)
for i in range(4):
    idx = np.nonzero(Igem[i] <= Nmax)[0]
    if len(idx) > 0:
        ax1.errorbar(ns[idx], Igem[i,idx], yerr=[Igem[i,idx]-Imin[i,idx], Imax[i,idx]-Igem[i,idx]], 
                     fmt=linestyles[i], label=labels[i], marker='o', markersize=4, capsize=3)
ax1.legend()
ax1.set_xscale('linear')
ax1.set_yscale('log')
ax1.set_xlabel(r"$n$")
ax1.set_ylabel(r"Number of iterations")

ax2 = plt.subplot(1,2,2)
for i in range(4):
    idx = np.nonzero(Igem[i] <= Nmax)[0]
    if len(idx) > 0:
        ax2.errorbar(ns[idx], Tgem[i,idx], yerr=[Tgem[i,idx]-Tmin[i,idx], Tmax[i,idx]-Tgem[i,idx]], 
                     fmt=linestyles[i], label=labels[i], marker='o', markersize=4, capsize=3)
ax2.legend()
ax2.set_xscale('linear')
ax2.set_yscale('log')
ax2.set_xlabel(r"$n$")
ax2.set_ylabel(r"Time (sec)")

plt.tight_layout(h_pad=0.5, w_pad=2)
#plt.savefig("C:\github\\accbpg\\figures\\Dopt_compareFW_m80e_8.pdf", bbox_inches="tight")

NameError: name 'Teps' is not defined